In [1]:
print("Hello")

Hello


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import re

In [7]:
#Data extraction
import numpy as np 
import pandas as pd

In [16]:
codingProfileData = pd.read_csv("coding-profile-data.csv")
codingProfileData

,Name,Codechef,Codeforces,Spoj,Leetcode,Gfg
0,sanchita,sanchita170676,sanchita170676,lakshay0510,sanchitamishra170676,sanchitamishra170676
1,Hem Kishan,vyas_7275,hem_7275,hem_7275,hem_7275,hem_7275


In [21]:
for ind in codingProfileData.index:
    print(codingProfileData.loc[ind,"Name"])

sanchita
Hem Kishan


In [5]:
class ProfileScraper():
    def __init__(self):
        print ("here")
        
        options = Options()
        options.add_argument('--headless')
        self.driver = webdriver.Chrome(options=options)
        
        self.profiles = {
            "codechef": {"https://www.codechef.com/users/sanchita170676": "//h5[contains(text(), 'Fully Solved')]"},
            "codeforces": {"https://codeforces.com/profile/sanchita170676": "//div[contains(@class, '_UserActivityFrame_counterValue')]"},
            "spoj": {"https://www.spoj.com/users/lakshay0510/": "//dl[contains(@class, 'profile-info-data')]/dd"},
            "leetcode": {"https://leetcode.com/sanchitamishra170676/": "//div[contains(@class, 'font-medium text-label-1')]"},
            "gfg": {"https://auth.geeksforgeeks.org/user/sanchitamishra170676/practice": "//span[contains(@class, 'score_card_value')]"},
        }
#         print(self.profiles)
        print( self.getProfiles())
        
        
    
    def getProfiles(self):
        print("here2")
        totalQuestions = 0
        for platform, data in self.profiles.items():
            for profile, query in data.items():
#                 print(profile," -> ", query)
                questions = self.runQuery(profile, query)
                print(profile, " ", questions, end="\n")
                totalQuestions += int(questions)
        return totalQuestions
    
    
    def runQuery(self, profile, query):
        try:
            self.driver.get(profile)
            element = WebDriverWait(self.driver, 5).until(
                EC.presence_of_element_located((By.XPATH, query))
            )
            questions = int(re.search(r'\d+', element.text).group())
            return questions
        except:
            return '0'
        


In [6]:
ProfileScraper()

here
here2
https://www.codechef.com/users/sanchita170676   131
https://codeforces.com/profile/sanchita170676   38
https://www.spoj.com/users/lakshay0510/   10
https://leetcode.com/sanchitamishra170676/   136
https://auth.geeksforgeeks.org/user/sanchitamishra170676/practice   127
442
